In [11]:
# Import required modules and variables
import pandas as pd
import requests
import pprint
from config import username, password, host, port, database

In [16]:
questionsfile = '../resources/QuestionsList.csv'
qs_df = pd.read_csv(questionsfile, delimiter=",")
qs_df.head()

,QuestionNum,Question
0,Q1,"""I would never audition to be on a game show."""
1,Q2,"""I am not much of a flirt."""
2,Q3,"""I have to psych myself up before I am brave e..."
3,Q4,"""I would hate living with room mates."""
4,Q5,"""I mostly listen to people in conversations."""


In [2]:
qcsvfile = '../resources/data.csv'
q_df = pd.read_csv(qcsvfile, delimiter="\t")
q_df.head()

,Q1A,Q1I,Q1E,Q2A,Q2I,Q2E,Q3A,Q3I,Q3E,Q4A,...,Q91E,country,dateload,introelapse,testelapse,surveyelapse,gender,engnat,age,IE
0,5,51,7107,3,91,2522,1,56,6180,2,...,4609,US,2019-02-20 17:35:52,1,461,16,2,1,23,3
1,5,39,6354,5,13,3092,1,12,5243,5,...,10409,AU,2019-02-20 17:46:32,21,467,15,1,2,25,2
2,3,17,5397,4,35,2747,5,40,5262,3,...,2691,BR,2019-02-20 18:10:24,56,306,17,1,2,19,1
3,5,41,3055,2,14,3348,1,13,5141,1,...,3697,CZ,2019-02-20 18:16:21,2,287,14,1,1,23,1
4,1,76,2542,2,54,1878,1,15,5637,1,...,1662,CA,2019-02-20 18:21:49,2,325,12,1,1,18,2


# (TRANSFORMATION): change date/time on startInterval and jobTimestamp

In [3]:
q_df.rename(columns ={'country':'COUNTRY', 
                                 'dateload':'DATELOAD',
                                 'introelapse':'INTROELAPSE',
                                 'testelapse':'TESTELAPSE',
                                 'surveyelapse':'SURVEYELAPSE',
                                 'gender':'GENDER',
                                 'engnat':'ENGNAT',
                                 'age':'AGE',
                                 'IE':'INTROVERT_EXTROVERT'}, inplace=True)
q_df.head()

,Q1A,Q1I,Q1E,Q2A,Q2I,Q2E,Q3A,Q3I,Q3E,Q4A,...,Q91E,COUNTRY,DATELOAD,INTROELAPSE,TESTELAPSE,SURVEYELAPSE,GENDER,ENGNAT,AGE,INTROVERT_EXTROVERT
0,5,51,7107,3,91,2522,1,56,6180,2,...,4609,US,2019-02-20 17:35:52,1,461,16,2,1,23,3
1,5,39,6354,5,13,3092,1,12,5243,5,...,10409,AU,2019-02-20 17:46:32,21,467,15,1,2,25,2
2,3,17,5397,4,35,2747,5,40,5262,3,...,2691,BR,2019-02-20 18:10:24,56,306,17,1,2,19,1
3,5,41,3055,2,14,3348,1,13,5141,1,...,3697,CZ,2019-02-20 18:16:21,2,287,14,1,1,23,1
4,1,76,2542,2,54,1878,1,15,5637,1,...,1662,CA,2019-02-20 18:21:49,2,325,12,1,1,18,2


# 5. (LOAD) into database
Use the values pulled in from config.py to construct the database connection string.  Connect to the database.  Write the data frames to the database tables

In [12]:
from sqlalchemy import create_engine

# Define the engine string to be used for connecting to your database using the 
#  environment variables found in config.py
connection_string = f'{username}:{password}@{host}:{port}/{database}'
engine = create_engine(f'postgresql://{connection_string}')

In [13]:
connection_string

'postgres:B00tCAmp8@database-bootcamp.cs5m4ix6jkw1.us-west-1.rds.amazonaws.com:5432/projectDB'

In [10]:
# Write questionnaire to database
q_df.to_sql(name='questionnaire', con=engine, if_exists='replace', index=False)

In [15]:
# Verify that there is data in finance_trends
pd.read_sql_query('select * from questionnaire', con=engine).head()

,Q1A,Q1I,Q1E,Q2A,Q2I,Q2E,Q3A,Q3I,Q3E,Q4A,...,Q91E,COUNTRY,DATELOAD,INTROELAPSE,TESTELAPSE,SURVEYELAPSE,GENDER,ENGNAT,AGE,INTROVERT_EXTROVERT
0,5,51,7107,3,91,2522,1,56,6180,2,...,4609,US,2019-02-20 17:35:52,1,461,16,2,1,23,3
1,5,39,6354,5,13,3092,1,12,5243,5,...,10409,AU,2019-02-20 17:46:32,21,467,15,1,2,25,2
2,3,17,5397,4,35,2747,5,40,5262,3,...,2691,BR,2019-02-20 18:10:24,56,306,17,1,2,19,1
3,5,41,3055,2,14,3348,1,13,5141,1,...,3697,CZ,2019-02-20 18:16:21,2,287,14,1,1,23,1
4,1,76,2542,2,54,1878,1,15,5637,1,...,1662,CA,2019-02-20 18:21:49,2,325,12,1,1,18,2


In [20]:
qs_df.head()

,QuestionNum,Question
0,Q1,"""I would never audition to be on a game show."""
1,Q2,"""I am not much of a flirt."""
2,Q3,"""I have to psych myself up before I am brave e..."
3,Q4,"""I would hate living with room mates."""
4,Q5,"""I mostly listen to people in conversations."""


In [24]:
# Write QuestionsList to database
qs_df.to_sql(name='questionslist', con=engine, if_exists='replace', index=False)

In [26]:
pd.read_sql_query('select * from questionslist', con=engine).head()

,QuestionNum,Question
0,Q1,"""I would never audition to be on a game show."""
1,Q2,"""I am not much of a flirt."""
2,Q3,"""I have to psych myself up before I am brave e..."
3,Q4,"""I would hate living with room mates."""
4,Q5,"""I mostly listen to people in conversations."""
